# User input imputation

In [ ]:
import pickle
from pathlib import Path

## 1. Generate data with NaN

In [ ]:
# Your code here... Generate a dataset with 20-30% NaN values randomly distributed in the 'biometric' features.

## 2. Train imputer

In [ ]:
# Your code here... Train a SciKit-Learn imputer to fill the missing values.

## 3. Evaluate

In [ ]:
# Your code here... Come up with a strategy to measure how good the imputation is.

## 4. Save

In [ ]:
Path('../models').mkdir(exist_ok=True)

with open('../models/imputer.pkl', 'wb') as output_file:
    pickle.dump(imputer, output_file)